In [25]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , BatchNormalization

In [26]:
CIFAR10 = keras.datasets.cifar10

(x_train , y_train) , (x_test , y_test ) = CIFAR10.load_data()
x_train , x_test = x_train/255.0 , x_test/255.0

In [27]:
x_train[2].shape

(32, 32, 3)

# Adding batch size

In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train , y_train)).batch(32).shuffle(buffer_size=1000)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test , y_test)).batch(32)

In [29]:
x_train.shape

(50000, 32, 32, 3)

# Dropout + Batch normalization

In [42]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv2D_1 = Conv2D(32 , (3,3) , activation="relu" , padding="same")
    self.conv2D_2 = Conv2D(32 , (3,3) , activation="relu" , padding="same")
    self.conv2D_3 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.conv2D_4 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.conv2D_5 = Conv2D(128 , (3,3) , activation="relu" , padding="same")
    self.conv2D_6 = Conv2D(128 , (3,3) , activation="relu" , padding="same")
    self.maxpool2D = MaxPooling2D()
    self.dropout = Dropout(0.2)
    self.dropout1 = Dropout(0.3)
    self.dropout2 = Dropout(0.4)
    self.dropout3 = Dropout(0.5)
    self.flatten = Flatten()
    self.dense1 = Dense(128 , activation="relu")
    self.dense = Dense(10 , activation="softmax")
    self.batchnorm1 = BatchNormalization()
    self.batchnorm2 = BatchNormalization()
    self.batchnorm3 = BatchNormalization()
    self.batchnorm4 = BatchNormalization()
    self.batchnorm7 = BatchNormalization()


  def call(self , x):
    x = self.conv2D_1(x)
    x = self.batchnorm1(x)
    x = self.conv2D_2(x)
    x = self.batchnorm2(x)
    x = self.maxpool2D(x)
    x = self.dropout(x)

    x = self.conv2D_3(x)
    x = self.batchnorm3(x)
    x = self.conv2D_4(x)
    x = self.batchnorm4(x)
    x = self.maxpool2D(x)
    x = self.dropout1(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.batchnorm7(x)
    x = self.dropout3(x)
    x = self.dense(x)
    return x

model = MyModel()

In [43]:
# Main loss function
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

In [44]:
# Metrics

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# Train

In [45]:
@tf.function
def train(images , labels):
    with tf.GradientTape() as tape :
      prediction = model(images)
      loss = loss_function(y_true=labels , y_pred=prediction)

    gradients = tape.gradient(loss , model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars= zip(gradients , model.trainable_variables))

    train_loss(loss)
    train_acc(labels, prediction)

In [46]:
@tf.function
def test(images , labels):
  prediction = model(images)
  loss = loss_function(y_true=labels , y_pred=prediction)
  test_loss(loss)
  test_acc(labels , prediction)

In [47]:
epochs = 5
for epoch in range(epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  #train
  for images , labels in train_dataset :
    train(images , labels)

  # test
  for images , labels in test_dataset :
    test(images , labels)


  print("epoch:" , epoch + 1  ,
        f"Train Loss: {train_loss.result()}" ,
        f"Train Acc: {train_acc.result()}" ,
        f"Test loss: {test_loss.result()}",
        f"Test Acc: {test_acc.result()}")

epoch: 1 Train Loss: 1.2753368616104126 Train Acc: 0.5448600053787231 Test loss: 0.9129331707954407 Test Acc: 0.6877999901771545
epoch: 2 Train Loss: 0.7675131559371948 Train Acc: 0.7348600029945374 Test loss: 0.7425907850265503 Test Acc: 0.7479000091552734
epoch: 3 Train Loss: 0.5786282420158386 Train Acc: 0.8008400201797485 Test loss: 0.7248903512954712 Test Acc: 0.7488999962806702
epoch: 4 Train Loss: 0.45234641432762146 Train Acc: 0.8482800126075745 Test loss: 0.8509339094161987 Test Acc: 0.7466999888420105
epoch: 5 Train Loss: 0.3482794165611267 Train Acc: 0.8826199769973755 Test loss: 0.823412299156189 Test Acc: 0.7635999917984009
